In [1]:
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error


In [2]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

In [3]:
df_train=pd.read_parquet("Data/green_tripdata_2021-01.parquet")
df_val=pd.read_parquet("Data/green_tripdata_2021-02.parquet")

In [4]:
#get trip duration

df_train["lpep_dropoff_datetime"]=pd.to_datetime(df_train["lpep_dropoff_datetime"])

df_train["lpep_pickup_datetime"]=pd.to_datetime(df_train["lpep_pickup_datetime"])

df_train["trip_duration"]=df_train["lpep_dropoff_datetime"]-df_train["lpep_pickup_datetime"]


In [5]:
#convert to duration in minutes
df_train["trip_duration"]=df_train["trip_duration"].apply(lambda x: x.total_seconds()/60)

In [6]:
#We are going to only consider trips with duration between 1 and 60 minutes

df_train=df_train[df_train["trip_duration"].between(1,60)]

In [7]:
#Select what categorical and numerical variables we are going to use

categorical=["PULocationID","DOLocationID"]
numerical=["trip_distance"]


In [8]:
#one-hot encoding to categorical variables

df_train[categorical]=df_train[categorical].astype(str)

In [10]:
#Create dict from df
train_dicts=df_train[categorical+numerical].to_dict(orient="records")

dv=DictVectorizer()
X_train=dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
#target variables
target="trip_duration"
y_train=df_train[target].values


In [20]:
df_val["lpep_dropoff_datetime"]=pd.to_datetime(df_val["lpep_dropoff_datetime"])

df_val["lpep_pickup_datetime"]=pd.to_datetime(df_val["lpep_pickup_datetime"])

df_val["trip_duration"]=df_val["lpep_dropoff_datetime"]-df_val["lpep_pickup_datetime"]

#convert to duration in minutes
df_val["trip_duration"]=df_val["trip_duration"].apply(lambda x: x.total_seconds()/60)

#We are going to only consider trips with duration between 1 and 60 minutes

df_val=df_val[df_val["trip_duration"].between(1,60)]


X_val=df_val.drop("trip_duration", axis=1)
y_val = df_val[target].values

In [19]:
#linear regression model
with mlflow.start_run():
    mlflow.set_tag("Developer", "FF")
    mlflow.log_param("train_data_path","January")
    mlflow.log_param("val_data_path","february")
    alpha=0.3
    mlflow.log_param("alpha",alpha)
    lr=Lasso(alpha)
    lr.fit(X_train, y_train)
    #Apply model
    y_pred=lr.predict(X_train)

    #RMSE of our baseline model
    RMSE=mean_squared_error(y_train, y_pred, squared=False)

    mlflow.log_metric("rmse", RMSE)

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()
mlflow.xgboost.autolog()

with mlflow.start_run():

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    #mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlmodel = LinearSVR()
    mlmodel.fit(X_train, y_train)

    y_pred = mlmodel.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
        

2022/05/23 23:07:19 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
d:\Anaconda\envs\mlops_zoomcamp\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Anaconda\envs\mlops_zoomcamp\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearSVR was fitted without feature names
  warnings.warn(


TypeError: float() argument must be a string or a number, not 'Timestamp'